In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
# --- Configura o Selenium para rodar em modo "headless" (sem abrir janela)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Remova essa linha se quiser ver o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# --- Inicializa o navegador
driver = webdriver.Chrome(options=chrome_options)

In [3]:
# --- URL do QR Code (exemplo real da nota)
url_qrcode = "http://nfe.sefaz.ba.gov.br/servicos/nfce/modulos/geral/NFCEC_consulta_chave_acesso.aspx?p=29250306057223048450650150000775861151017568|2|1|1|8DF25CB77517F73F80677AE283AEE7DFCCA38282"
driver.get(url_qrcode)

In [4]:
# --- Aguarda a página carregar completamente
time.sleep(5)  # Pode ajustar dependendo da conexão

# --- Extrai o HTML renderizado
html = driver.page_source
driver.quit()

In [5]:
# --- Faz o parsing com BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

In [6]:
# --- Localiza a tabela com id="tabResult"
tabela = soup.find("table", {"id": "tabResult"})

In [7]:
produtos = []

if tabela:
    linhas = tabela.find_all("tr")
    for linha in linhas:
        colunas = linha.find_all("td")
        if len(colunas) >= 2:
            # Descrição do produto
            descricao_tag = colunas[0].find("span", class_="txtTit")
            descricao = descricao_tag.text.strip() if descricao_tag else ""

            # Código do produto (entre parênteses)
            codigo_tag = colunas[0].find("span", class_="RCod")
            codigo = ""
            if codigo_tag:
                match = re.search(r"\d+", codigo_tag.text)
                if match:
                    codigo = match.group()

            # Valor total
            valor_tag = colunas[1].find("span", class_="valor")
            valor_total = valor_tag.text.strip() if valor_tag else ""

            # Adiciona ao DataFrame
            produtos.append({
                "Descrição": descricao,
                "Código": codigo,
                "Valor Total": valor_total
            })

    # Cria o DataFrame
    df = pd.DataFrame(produtos)
    print(df)
else:
    print("❌ Tabela 'tabResult' não encontrada.")

                Descrição   Código Valor Total
0        TOMATE SALADA kg     8186        3,78
1            BETERRABA kg     8134        0,85
2           CEBOLA NAC kg     7832        1,31
3         BANANA TERRA kg    62885        2,05
4               PITAYA kg  1062316        8,59
..                    ...      ...         ...
103    TAPIOCA LIMIAR 1kg  1059242        7,99
104  DORITOS NACHO 210G Q  1150692       20,90
105   WAFER BAUD 70G BRIG  1200515        5,67
106  SORV DIAM/LAKA 1,5LT  1102715       25,90
107  REF COCA COL 6X350ML  1137357       19,49

[108 rows x 3 columns]


In [8]:
produtos = []

if tabela:
    linhas = tabela.find_all("tr")
    for linha in linhas:
        colunas = linha.find_all("td")
        if len(colunas) >= 2:
            # Descrição
            descricao_tag = colunas[0].find("span", class_="txtTit")
            descricao = descricao_tag.text.strip() if descricao_tag else ""

            # Código
            codigo_tag = colunas[0].find("span", class_="RCod")
            codigo = ""
            if codigo_tag:
                match = re.search(r"\d+", codigo_tag.text)
                if match:
                    codigo = match.group()

            # Extrai o restante do texto do td esquerdo
            info_extra = colunas[0].text

            # Regex para quantidade, unidade e valor unitário
            qtd_match = re.search(r"Qtde:\s*([\d,\.]+)", info_extra)
            un_match = re.search(r"UN:\s*([A-Za-z]+)", info_extra)
            vl_unit_match = re.search(r"Vl\. Unit\.\s*:\s*([\d,]+)", info_extra)

            quantidade = qtd_match.group(1) if qtd_match else ""
            unidade = un_match.group(1) if un_match else ""
            valor_unitario = vl_unit_match.group(1) if vl_unit_match else ""

            # Valor total
            valor_tag = colunas[1].find("span", class_="valor")
            valor_total = valor_tag.text.strip() if valor_tag else ""

            produtos.append({
                "Descrição": descricao,
                "Código": codigo,
                "Quantidade": quantidade,
                "Unidade": unidade,
                "Valor Unitário": valor_unitario,
                "Valor Total": valor_total
            })

    df = pd.DataFrame(produtos)
    print(df)
else:
    print("❌ Tabela 'tabResult' não encontrada.")

                Descrição   Código Quantidade Unidade Valor Unitário  \
0        TOMATE SALADA kg     8186                 Kg           7,49   
1            BETERRABA kg     8134                 Kg           3,99   
2           CEBOLA NAC kg     7832                 Kg           3,99   
3         BANANA TERRA kg    62885                 Kg           3,49   
4               PITAYA kg  1062316                 Kg           17,9   
..                    ...      ...        ...     ...            ...   
103    TAPIOCA LIMIAR 1kg  1059242                 Un           7,99   
104  DORITOS NACHO 210G Q  1150692                 PC           20,9   
105   WAFER BAUD 70G BRIG  1200515                 Un           1,89   
106  SORV DIAM/LAKA 1,5LT  1102715                 Pt           25,9   
107  REF COCA COL 6X350ML  1137357                 Fd          19,49   

    Valor Total  
0          3,78  
1          0,85  
2          1,31  
3          2,05  
4          8,59  
..          ...  
103      